In [3]:
import sqlite3
import pandas as pd

In [4]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [5]:
cur = con.cursor()

In [6]:
check_query = 'SELECT * FROM Facilities'

In [7]:
check_result = cur.execute(check_query).fetchall()

In [8]:
print(check_result)

[(0, 'Tennis Court 1', 5, 25, 10000, 200), (1, 'Tennis Court 2', 5, 25, 8000, 200), (2, 'Badminton Court', 0, 15.5, 4000, 50), (3, 'Table Tennis', 0, 5, 320, 10), (4, 'Massage Room 1', 9.9, 80, 4000, 3000), (5, 'Massage Room 2', 9.9, 80, 4000, 3000), (6, 'Squash Court', 3.5, 17.5, 5000, 80), (7, 'Snooker Table', 0, 5, 450, 15), (8, 'Pool Table', 0, 5, 400, 15)]


#### Q10: Produce a list of facilities with a total revenue less than 1000.
##### The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [9]:
Q10_query = '''
SELECT 
    Facilities.name AS facility_name,
    SUM( CASE 
            WHEN Bookings.memid = 0
                THEN Bookings.slots * Facilities.guestcost 
            ELSE Bookings.slots * membercost 
        END) AS total_revenue 
FROM Bookings 
JOIN Facilities ON Facilities.facid = Bookings.facid
GROUP BY Facilities.name
HAVING total_revenue < 1000
ORDER BY total_revenue DESC;
'''

In [10]:
Q10_result = cur.execute(Q10_query).fetchall()

In [11]:
print(Q10_result)

[('Pool Table', 270), ('Snooker Table', 240), ('Table Tennis', 180)]


#### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [13]:
Q11_query = '''
SELECT 
    (M1.surname || ', '|| M1.firstname) AS Member,
    COALESCE((M2.surname || ', '|| M2.firstname), 'None') AS Recommender
FROM Members AS M1
    LEFT JOIN Members AS M2
        ON M1.recommendedby = M2.memid
WHERE M1.firstname NOT LIKE 'GUEST'
ORDER BY Member ASC;
'''

In [14]:
Q11_result = cur.execute(Q11_query).fetchall()

In [15]:
Q11_col_names = ['Member', 'Recommender']
Q11_frame = pd.DataFrame(Q11_result, columns = Q11_col_names)

In [16]:
print(Q11_frame)

                      Member         Recommender
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10          Genting, Matthew     Butters, Gerald
11                Hunt, John  Purview, Millicent
12              Jones, David    Joplette, Janice
13            Jones, Douglas        Jones, David
14          Joplette, Janice       Smith, Darren
15           Mackenzie, Anna       Smith, Darren
16             Owen, Charles       Smith, Darren
17             Pinker, David     Farrell, Jemima
18        Purview, Millicent        Smith, Tracy
19               Row

#### Q12: Find the facilities with their usage by member, but not guests

In [17]:
Q12_query = '''
SELECT 
    Facilities.name AS facility_name,
    (Members.surname || ', ' || Members.firstname) AS member_name, 
    SUM(Bookings.slots) AS slots_booked
FROM Bookings
    JOIN Facilities
        ON Facilities.facid = Bookings.facid
    JOIN Members
        ON Members.memid = Bookings.memid
WHERE Bookings.memid > 0
GROUP BY facility_name, member_name
ORDER BY Facilities.name, Members.surname ASC;
'''

In [60]:
Q12_result = cur.execute(Q12_query).fetchall()

In [61]:
Q12_col_names = ['Facility_Name', 'Member_Name', 'Slots_Booked']
Q12_frame = pd.DataFrame(Q12_result, columns = Q12_col_names)

In [62]:
print(Q12_frame)

       Facility_Name       Member_Name  Slots_Booked
0    Badminton Court   Bader, Florence            27
1    Badminton Court       Baker, Anne            30
2    Badminton Court    Baker, Timothy            21
3    Badminton Court       Boothe, Tim            36
4    Badminton Court   Butters, Gerald            63
..               ...               ...           ...
197   Tennis Court 2     Smith, Darren            57
198   Tennis Court 2       Smith, Jack             3
199   Tennis Court 2      Smith, Tracy             6
200   Tennis Court 2  Stibbons, Ponder            96
201   Tennis Court 2     Tracy, Burton             9

[202 rows x 3 columns]


#### Q13: Find the facilities usage by month, but not guests

In [77]:
Q13_query = '''
SELECT 
    CASE 
        WHEN Bookings.starttime LIKE '____-07-%' THEN 'July'
        WHEN Bookings.starttime LIKE '____-08-%' THEN 'August'
        WHEN Bookings.starttime LIKE '____-09-%' THEN 'September'
        ELSE 'Other' 
    END AS month, 
Facilities.name AS facility_name,
SUM(Bookings.slots) AS slots_booked
FROM Bookings
JOIN Facilities ON Facilities.facid = Bookings.facid
WHERE Bookings.memid > 0
GROUP BY facility_name, month
ORDER BY Bookings.starttime ASC;
'''

In [78]:
Q13_result = cur.execute(Q13_query).fetchall()

In [79]:
Q13_col_names = ['Month', 'Facility_Name', 'Slots_Booked']
Q13_frame = pd.DataFrame(Q13_result, columns = Q13_col_names)

In [80]:
print(Q13_frame)

        Month    Facility_Name  Slots_Booked
0        July   Massage Room 1           166
1        July       Pool Table           110
2        July     Table Tennis            98
3        July    Snooker Table           140
4        July   Tennis Court 1           201
5        July     Squash Court            50
6        July  Badminton Court           165
7        July   Tennis Court 2           123
8        July   Massage Room 2             8
9      August   Massage Room 1           316
10     August   Massage Room 2            18
11     August       Pool Table           303
12     August   Tennis Court 2           345
13     August  Badminton Court           414
14     August    Snooker Table           316
15     August     Table Tennis           296
16     August     Squash Court           184
17     August   Tennis Court 1           339
18  September   Massage Room 1           402
19  September    Snooker Table           404
20  September     Table Tennis           400
21  Septem